1. Data Preparation.ipynb

Purpose: Load, clean, and preprocess the dataset.
Contents:
Import and inspect data.
Visualize samples of images and classes.
Preprocess images (resize, normalize, etc.).
Split data into training, validation, and test sets.
Save processed data to disk (if needed) to avoid re-running preprocessing steps.

In [ ]:
from PIL import Image
import os

def resize_images(input_dir, output_dir, size=(128, 128)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder in os.listdir(input_dir):
        folder_path = os.path.join(input_dir, folder)
        output_folder_path = os.path.join(output_dir, folder)
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)

        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            img = Image.open(image_path)
            img = img.resize(size)
            img.save(os.path.join(output_folder_path, image_file))

resize_images('archive/crop', 'dataset_resized')